In [67]:
import os
import pandas as pd
import seaborn as sns
import numpy as np


%matplotlib inline

INPUT_FILE = os.path.join("input", "titanic.csv")
REPORT_FILE = os.path.join("output","report.html")
GENERATE_REPORT = False
EXPORT_FILE = os.path.join("output","titanic_processed.csv")

raw_data = pd.read_csv(INPUT_FILE)
raw_data.head()

#from pandas_profiling import ProfileReport

#profile = ProfileReport(raw_data)
#profile.to_file(REPORT_FILE)


#Counting People in different classes
data = raw_data.copy()
data["alive"] = data["survived"].map({0: False, 1: True})
data["survived"].value_counts()

data["class"] = data["pclass"].astype(pd.CategoricalDtype(ordered=True))
data["class"] = data["class"].map({1: 'first', 2: 'second', 3: 'third'})

data["class"].value_counts(sort=False)

# Data manipulation of Cabin data
data["cabin"].value_counts()
data["deck"] = data["cabin"].str["0"]
data["deck"].value_counts()

x = "B96 B98"
x[0:3]

# Calculation of lone travelling persons
(data["parch"] + data["sibsp"]).value_counts()
data["alone"] = -(data["parch"] + data["sibsp"]).astype(bool)
data["alone"].value_counts()

pd.crosstab(data["parch"], data["alone"])

# Calculation of age/gender groups
# 1. Children 2. Female 3. Male

def women_children (passenger):
    if passenger["age"] < 16:
        return "child"
    #elif passenger["age"] >= 14 and passenger["sex"] == "male":
    #    return "male"
    #elif passenger["age"] >= 14 and passenger["sex"] == "female":
    #    return "female"
    else:
        return passenger["sex"]



p1 = {"age": 12, "sex": "male"}
p1 = {"age": 30, "sex": "female"}

data["who"] = data.apply(women_children, axis=1)
data["who"].value_counts(sort= False, normalize=True).round(3) * 100

data["wc"] = data["who"].map({"male": "m", "female": "w/c", "children": "w/c"})
pd.crosstab(data["who"], data["wc"])


wc,m,w/c
who,,
female,0,271
male,537,0


In [68]:
#Data anonymization

critical_vars = ["name", "ticket", "cabin"]
data = data.drop(critical_vars, axis =1)
data.head(1)



,survived,pclass,sex,age,sibsp,parch,fare,embarked,alive,class,deck,alone,who,wc
0,0,3,male,22.0,1,0,7.25,S,False,third,NaN,False,male,m


In [69]:
data.to_csv(EXPORT_FILE)